In [1]:
## It's the first time I'm using google colab.
## Just checking it works fine!

print ('hello world')

hello world


**Sentiment Analysis Using RNN**

In [2]:
import pandas as pd

**Uploading Dataset from Device**

In [4]:

from google.colab import files

In [10]:
uploaded = files.upload()

Saving Reviews.csv to Reviews.csv


In [11]:
import io

**Loading Dataset**

In [12]:
df = pd.read_csv(io.BytesIO(uploaded['Reviews.csv']))

In [13]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


**Since there is no column in our dataset to denote if review is positive or negative, we generate a new column named 'Label'**

**Where 'Label' is a Binary Variable (1 denoting Positive Review and 0 denoting Negative Review)**

Label = 1 when score > 3

Label = 0 when score <= 3

In [15]:
import numpy as np

In [17]:
## Creating a new column 'label'

df['label'] = np.where(df['Score']>3, 1, 0)

In [18]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,label
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [24]:
## Let's define our dependent and independent variable

x = df['Text']
y = df['label']

In [25]:
## Train-test Split

from sklearn.model_selection import train_test_split

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [27]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (426340,)
x_test shape: (142114,)
y_train shape: (426340,)
y_test shape: (142114,)


**Let's start making the model**

Step 1: Text Preprocessing

In [28]:
## Importing the libraries

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer

In [30]:
## Word tokenizer

tokenizer = Tokenizer(num_words= 10000, lower=True)                    ## considers top 10000 of most used words

## Converting words to index
tokenizer.fit_on_texts(x_train)


In [31]:
## Next, we'll convert it to sequence of numbers

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [33]:
vocab_size = len(tokenizer.word_index) + 1

In [36]:
## pad_sequences: These are used to keep the size of the vector fixed
from keras.preprocessing.sequence import pad_sequences

max_len = 200         ## let's set a max limit of 200 words in a sentence
x_train = pad_sequences(x_train, padding='post', maxlen=max_len)
x_test = pad_sequences(x_test, padding='post', maxlen=max_len)

In [37]:
print(x_train[3,:])

[   2   91   25  608   14   22   17    9  412   11  802    2  332  270
  227    1   61  129    3    1  605  412 1341   43 7183 2616 1879 6668
 1555 1071   93    1 1655 2047   20    1  241    7    4 2865    3   16
 4469    6 5250   37  537   79   20   68  148  196  664    5    1 1742
 1683    7    1 4218    2 1058   63 1569  140    7    1 1655  384  317
    2   91   25  383   12    9   39    2  100   17  143 1975  227    7
    9  166    2   17  359   29    5  178   64    6   15   36   19   26
  799 1655  384   16  690    7 4469  115    9  276   30   72   39    2
   91  535  722    5    1 5691    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [38]:
## One hot encoding for 'y' variable

from keras.utils import to_categorical

num_categories = 2
y_train = to_categorical(y_train, num_categories)
y_test = to_categorical(y_test, num_categories)

Step 2: Training the Model

In [39]:
## Importing libraries

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN
from keras import optimizers

In [41]:
## RNN takes 3 inputs. So we reshape our data into 3 dimensions

x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)
print(x_train.shape)

x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1], 1)
print(x_test.shape)

(426340, 200, 1)
(142114, 200, 1)


In [52]:
## Let's define our RNN model

def rnn_model():
  model = Sequential()
  # adding the layers in model
  model.add(SimpleRNN(100, input_shape=(max_len,1), return_sequences=False))       #number of neurons=100
  model.add(Dense(num_categories))                         #fully connected layer
  model.add(Activation('softmax'))                         #calculates the probability
  model.summary()

  adam = optimizers.Adam(lr=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

In [46]:
try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

In [53]:
## Now, we will fit our data to the model

from scikeras.wrappers import KerasClassifier
#import scikeras

model = KerasClassifier(build_fn=rnn_model, epochs=3, batch_size=50)
model.fit(x_train, y_train)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 100)               10200     
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 10402 (40.63 KB)
Trainable params: 10402 (40.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/3
8527/8527 [==============================] - 456s 53ms/step - loss: 0.5291 - accuracy: 0.7808
Epoch 2/3
8527/8527 [==============================] - 474s 56ms/step - loss: 0.5277 - accuracy: 0.7808
Epoch 3/3
8527/8527 [==============================] - 474s 56ms/step - loss: 0.5278 - accuracy: 0.7809


KerasClassifier(
	model=None
	build_fn=<function rnn_model at 0x7bf493d7ff40>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=50
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=3
	class_weight=None
)

We can see from the above output that the accuracy and loss do not change much with epochs. So now, let's movee ahead with testing our model.

In [56]:
## Test Data

from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)
#y_val = np.argmax(y_test, axis=1)

print(accuracy_score(y_pred, y_test))

2843/2843 [==============================] - 73s 26ms/step
0.7801342584122606


We can see that our model performed equally well on Test data